<img src="../../images/banners/python-oop.png" width="600"/>

# <img src="../../images/logos/python.png" width="23"/> Multiple Inheritance


## <img src="../../images/logos/toc.png" width="20"/> Table of Contents 


---

Now that you’ve worked through an overview and some examples of `super()` and single inheritance, you will be introduced to an overview and some examples that will demonstrate how multiple inheritance works and how `super()` enables that functionality. 

## Multiple Inheritance using `super()`

There is another use case in which `super()` really shines, and this one isn’t as common as the single inheritance scenario. In addition to single inheritance, Python supports multiple inheritance, in which a subclass can inherit from multiple superclasses that don’t necessarily inherit from each other (also known as **sibling classes**). 

The image below shows a very simple multiple inheritance scenario, where one class inherits from two unrelated (sibling) superclasses:

<img src="./images/multiple-inheritance.svg" width="300"/>

To better illustrate multiple inheritance in action, here is some code for you to try out, showing how you can build a right pyramid (a pyramid with a square base) out of a `Triangle` and a `Square`:

<img src="./images/square-triangle-pyramid.svg" width="200"/>

In [3]:
# Here we declare that the Square class inherits from the Rectangle class
class Square:
    def __init__(self, length):
        self.length = length

    def area(self):
        return self.length ** 2

    def perimeter(self):
        return 4 * self.length

In [4]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height

    def area(self):
        return 0.5 * self.base * self.height

class RightPyramid(Triangle, Square):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

This example declares a `Triangle` class and a `RightPyramid` class that inherits from both `Square` and `Triangle`. 

You’ll see another `.area()` method that uses `super()` just like in single inheritance, with the aim of it reaching the `.perimeter()` and `.area()` methods defined all the way up in the `Rectangle` class. 

The problem, though, is that both superclasses (`Triangle` and `Square`) define a `.area()`. Take a second and think about what might happen when you call `.area()` on `RightPyramid`, and then try calling it like below:

In [5]:
pyramid = RightPyramid(2, 4)
pyramid.area()

AttributeError: 'RightPyramid' object has no attribute 'height'

Did you guess that Python will try to call `Triangle.area()`? This is because of something called the **method resolution order**.

<a class="anchor" id="method_resolution_order"></a>

### Method Resolution Order

The method resolution order (or **MRO**) tells Python how to search for inherited methods. This comes in handy when you’re using `super()` because the MRO tells you exactly where Python will look for a method you’re calling with `super()` and in what order.

Every class has an `.__mro__` attribute that allows us to inspect the order, so let’s do that:

In [6]:
RightPyramid.__mro__

(__main__.RightPyramid, __main__.Triangle, __main__.Square, object)

This tells us that methods will be searched first in `Rightpyramid`, then in `Triangle`, then in `Square`, then `Rectangle`, and then, if nothing is found, in `object`, from which all classes originate. 

The problem here is that the interpreter is searching for `.area()` in `Triangle` before `Square` and `Rectangle`, and upon finding `.area()` in `Triangle`, Python calls it instead of the one you want. Because `Triangle.area()` expects there to be a `.height` and a `.base` attribute, Python throws an `AttributeError`. 

Luckily, you have some control over how the MRO is constructed. Just by changing the signature of the `RightPyramid` class, you can search in the order you want, and the methods will resolve correctly: